In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ["PYTHONWARNINGS"] = "ignore"

## Import dependencies

In [3]:
import os
import pandas as pd
from IPython.display import display
from tqdm import tqdm

from configs import config
from configs.constants import ModelSetting
from utils.analyzers.stability_fairness_analyzer import StabilityFairnessAnalyzer
from utils.common_helpers import create_tuned_base_model, save_metrics_to_file
from utils.custom_classes.data_loader import ACSEmploymentDataset
from utils.custom_classes.generic_pipeline import GenericPipeline
from utils.analyzers.bias_analyzer import BiasAnalyzer

## Configs

In [ ]:
STATE = config.DATASET_CONFIG['state']
YEAR = config.DATASET_CONFIG['year']
DATASET_NAME = f"Folktables_{STATE}_{YEAR}"
EXPERIMENT_NAME = 'Hypothesis_Space'

SEX_priv = RACE_priv = str(1)
# N_ESTIMATORS = 200
N_ESTIMATORS = 10
PROTECTED_GROUPS = ['SEX','RAC1P']
PRIV_VALUES = [SEX_priv, RACE_priv]
TUNED_PARAMS_FILE_PATH = os.path.join('..', '..', 'results', 'models_tuning', 'tuning_results_Folktables_GA_2018_20221215__105658.csv')

## Models tuned hyper-parameters

In [ ]:
models_tuned_params_df = pd.read_csv(TUNED_PARAMS_FILE_PATH)
models_tuned_params_df

## Preprocess dataset

In [ ]:
dataset = ACSEmploymentDataset(state=[STATE], year=YEAR, root_dir=os.path.join('..', '..', 'data'), with_nulls=False)
dataset.X_data.head()

## Run experiments

In [ ]:
def create_base_pipeline(dataset, protected_groups, priv_values, model_seed):
    base_pipeline = GenericPipeline(dataset, protected_groups, priv_values)
    _ = base_pipeline.create_preprocessed_train_test_split(dataset, config.TEST_SET_FRACTION, seed=model_seed)

    print('\nProtected groups splits:')
    for g in base_pipeline.test_groups.keys():
        print(g, base_pipeline.test_groups[g].shape)

    return base_pipeline


def get_model_metrics(base_model, n_estimators, dataset, protected_groups, priv_values, model_seed,
                      dataset_name, base_model_name, exp_num=1):
    base_pipeline = create_base_pipeline(dataset, protected_groups, priv_values, model_seed)

    stability_fairness_analyzer = StabilityFairnessAnalyzer(ModelSetting.BATCH, n_estimators, base_model, base_model_name,
                                                            base_pipeline.X_train_val, base_pipeline.y_train_val,
                                                            base_pipeline.X_test, base_pipeline.y_test,
                                                            base_pipeline.protected_groups, base_pipeline.priv_values, base_pipeline.test_groups,
                                                            base_pipeline.target, dataset_name)

    save_results = False
    y_preds, variance_metrics_df = stability_fairness_analyzer.compute_metrics(save_results=save_results,
                                                                               result_filename=None,
                                                                               save_dir_path=None,
                                                                               make_plots=False)

    bias_analyzer = BiasAnalyzer(base_pipeline.X_test, base_pipeline.y_test,
                                 base_pipeline.protected_groups, base_pipeline.priv_values,
                                 base_pipeline.test_groups)
    dtc_res = bias_analyzer.compute_subgroups_metrics(y_preds,
                                                      save_results=False,
                                                      result_filename=None,
                                                      save_dir_path=None)
    bias_metrics_df = pd.DataFrame(dtc_res)

    metrics_df = pd.concat([variance_metrics_df, bias_metrics_df])
    result_filename = f'{EXPERIMENT_NAME}_Metrics_{dataset_name}_Experiment_{exp_num}_{base_model_name}'
    save_dir_path = os.path.join('..', '..', 'results', 'hypothesis_space')
    save_metrics_to_file(metrics_df, result_filename, save_dir_path)

    return metrics_df


In [ ]:
def run_experiment(exp_num, model_seed):
    for model_idx in tqdm(range(len(config.MODELS_CONFIG))):
        print('#' * 30, f' [Experiment {exp_num}] Analyze {config.MODELS_CONFIG[model_idx]["model_name"]} ', '#' * 30)
        model_seed += 1
        try:
            base_model = create_tuned_base_model(config.MODELS_CONFIG[model_idx]['model'],
                                                 config.MODELS_CONFIG[model_idx]['model_name'],
                                                 models_tuned_params_df)
            results_df = get_model_metrics(base_model, N_ESTIMATORS, dataset, PROTECTED_GROUPS, PRIV_VALUES,
                                           model_seed=model_seed,
                                           dataset_name=DATASET_NAME,
                                           base_model_name=config.MODELS_CONFIG[model_idx]['model_name'],
                                           exp_num=exp_num)
            print(f'\n[Experiment {exp_num}] Metrics confusion matrix:')
            display(results_df)
        except Exception as err:
            print(f'ERROR with {config.MODELS_CONFIG[model_idx]["model_name"]}: ', err)

        print('\n\n\n')


### Experiment 1

In [9]:
# TOD: add dataset as a parameter
run_experiment(exp_num=1, model_seed=100)

  0%|          | 0/3 [00:00<?, ?it/s]

##############################  [Experiment 1] Analyze XGBClassifier  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)


2022-12-16 10:23:59 abstract_stability_analyzer.py INFO    : Start testing of classifier 1 / 50



Protected groups splits:
SEX_RAC1P_priv (6609, 16)
SEX_RAC1P_dis (3662, 16)
SEX_priv (9901, 16)
SEX_dis (10270, 16)
RAC1P_priv (13217, 16)
RAC1P_dis (6954, 16)


2022-12-16 10:24:19 abstract_stability_analyzer.py INFO    : Classifier 1 / 50 was tested
2022-12-16 10:24:19 abstract_stability_analyzer.py INFO    : Start testing of classifier 2 / 50
2022-12-16 10:24:33 abstract_stability_analyzer.py INFO    : Classifier 2 / 50 was tested
2022-12-16 10:24:33 abstract_stability_analyzer.py INFO    : Start testing of classifier 3 / 50
2022-12-16 10:24:49 abstract_stability_analyzer.py INFO    : Classifier 3 / 50 was tested
2022-12-16 10:24:49 abstract_stability_analyzer.py INFO    : Start testing of classifier 4 / 50
2022-12-16 10:25:13 abstract_stability_analyzer.py INFO    : Classifier 4 / 50 was tested
2022-12-16 10:25:13 abstract_stability_analyzer.py INFO    : Start testing of classifier 5 / 50
2022-12-16 10:25:32 abstract_stability_analyzer.py INFO    : Classifier 5 / 50 was tested
2022-12-16 10:25:32 abstract_stability_analyzer.py INFO    : Start testing of classifier 6 / 50
2022-12-16 10:25:46 abstract_stability_analyzer.py INFO    : Classifie

...





##############################  [Experiment 1] Analyze MLPClassifier_1L_100  ##############################
Baseline X_train shape:  (80684, 16)
Baseline X_test shape:  (20171, 16)


2022-12-16 10:37:08 abstract_stability_analyzer.py INFO    : Start testing of classifier 1 / 50



Protected groups splits:
SEX_RAC1P_priv (6582, 16)
SEX_RAC1P_dis (3543, 16)
SEX_priv (9817, 16)
SEX_dis (10354, 16)
RAC1P_priv (13393, 16)
RAC1P_dis (6778, 16)


2022-12-16 10:39:54 abstract_stability_analyzer.py INFO    : Classifier 1 / 50 was tested
2022-12-16 10:39:54 abstract_stability_analyzer.py INFO    : Start testing of classifier 2 / 50
2022-12-16 10:42:44 abstract_stability_analyzer.py INFO    : Classifier 2 / 50 was tested
2022-12-16 10:42:44 abstract_stability_analyzer.py INFO    : Start testing of classifier 3 / 50
2022-12-16 10:45:36 abstract_stability_analyzer.py INFO    : Classifier 3 / 50 was tested
2022-12-16 10:45:36 abstract_stability_analyzer.py INFO    : Start testing of classifier 4 / 50
2022-12-16 10:48:28 abstract_stability_analyzer.py INFO    : Classifier 4 / 50 was tested
2022-12-16 10:48:28 abstract_stability_analyzer.py INFO    : Start testing of classifier 5 / 50
2022-12-16 10:50:40 abstract_stability_analyzer.py INFO    : Classifier 5 / 50 was tested
2022-12-16 10:50:40 abstract_stability_analyzer.py INFO    : Start testing of classifier 6 / 50
2022-12-16 10:51:12 abstract_stability_analyzer.py INFO    : Classifie

KeyboardInterrupt: 

### Experiment 2

In [ ]:
# run_experiment(exp_num=2, model_seed=200)